In [3]:
from __future__ import division, print_function
 
import numpy as np
import sympy as sym
import scipy.linalg
 
def lqr(A,B,Q,R):
    """Solve the continuous time lqr controller.
 
    dx/dt = A x + B u
 
    cost = integral x.T*Q*x + u.T*R*u
    """
    #ref Bertsekas, p.151
 
    #first, try to solve the ricatti equation
    X = np.matrix(scipy.linalg.solve_continuous_are(A, B, Q, R))
 
    #compute the LQR gain
    K = np.matrix(scipy.linalg.inv(R)*(B.T*X))
 
    eigVals, eigVecs = scipy.linalg.eig(A-B*K)
 
    return K, X, eigVals


A = np.array([[0, 0,  1,0],
              [0, 0,0,1],
              [0, 10,  0,0],
              [0,-20,0,0]])

B = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

Q = np.array([[1, 0, 0, 0],
              [0, 100, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1000]])

R = np.array([[0.001, 0, 0, 0],
              [0, 0.001, 0, 0],
              [0, 0, 0.001, 0],
              [0, 0, 0, 0.001]])
  

K, X, eigVals = lqr(A,B,Q,R)

print(K)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.16227765e+01 9.97916844e+00 3.26074449e+01 9.66328175e-03]
 [2.07796723e-03 2.97017349e+02 9.66328175e-03 1.00029697e+03]]


In [7]:
X = np.asarray(sym.symbols('x:' + str(4))) # 1.half= location, 2.half= velocity

print(np.dot(K,X).T)



[[0]
 [0]
 [31.622776533411*x0 + 9.97916844332367*x1 + 32.6074448506445*x2 + 0.00966328174824767*x3]
 [0.0020779672263365*x0 + 297.017348998508*x1 + 0.00966328174824767*x2 + 1000.29697320578*x3]]


In [2]:
"""
author: wolfgang flachberger (c)
08-04-2020
"""
import numpy as np
from numpy import sin,cos,pi
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.integrate import odeint

# constants
dt = 0.01
T = 10
g = 9.81
l = 1
m0 = 1 
m1 = 1
m2 = 1
m3 = 1
timesteps = int(T/dt)

#dimensions 
n = 401

# initial condition
X0 = [0,0.01,0,0]

# number of time points
p = 401

# time6 points
t = np.linspace(0,T,timesteps)


# dynamics         
def f(X,t):
    x0 = X[0]
    x1 = X[1]
    x2 = X[2]
    x3 = X[3]
    
    dx0_dt = x2
    dx1_dt = x3
    dx2_dt = (10*cos(x1) + x3**2)*sin(x1)/(1 + sin(x1)**2) -(31.622776533411*x0 + 9.97916844332367*x1 + 32.6074448506445*x2 + 0.00966328174824767*x3)
    dx3_dt = -(20 + x3**2*cos(x1))*sin(x1)/((1 + sin(x1)**2)) - (0.0020779672263365*x0 + 297.017348998508*x1 + 0.00966328174824767*x2 + 1000.29697320578*x3)
    
    X_dot = [dx0_dt, dx1_dt, dx2_dt, dx3_dt]
    return X_dot

# integration of dynamic equations 
X_t = odeint(f,X0,t)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=100, metadata=dict(artist='Me'), bitrate=1800)

x1 = X_t[:,0]
y1 = np.zeros(timesteps)

x2 = l*np.cos(X_t[:,1]) + x1
y2 = l*np.sin(X_t[:,1]) 


plt.plot(x4,y4)

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-3, 3), ylim=(-3, 3))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text

def animate(i):
    thisx = [ x1[i], x2[i]]
    thisy = [ y1[i], y2[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

anim = animation.FuncAnimation(fig, animate, range(1, len(X_t)),
                              interval=dt*timesteps, blit=True, init_func=init)
plt.show()

#anim.save('pendelwagen.mp4', writer=writer)




RuntimeError: Requested MovieWriter (ffmpeg) not available